In [3]:
import os
import numpy as np
from numpy import array
import matplotlib
import matplotlib.gridspec as gridspec
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
# from mockfactory import Catalog
from cosmoprimo.fiducial import DESI
from desilike.theories.galaxy_clustering import DirectPowerSpectrumTemplate, ShapeFitPowerSpectrumTemplate, StandardPowerSpectrumTemplate
from desilike.theories.galaxy_clustering import KaiserTracerPowerSpectrumMultipoles, TNSTracerPowerSpectrumMultipoles, FOLPSTracerPowerSpectrumMultipoles, FOLPSRCTracerPowerSpectrumMultipoles
from desilike.observables.galaxy_clustering import TracerPowerSpectrumMultipolesObservable
from desilike.emulators import EmulatedCalculator, Emulator, TaylorEmulatorEngine
from desilike.likelihoods import ObservablesGaussianLikelihood
from desilike.profilers import MinuitProfiler
from desilike.samples import Profiles
from desilike import setup_logging

setup_logging()  # for logging messages

# kbin = 'kbin2' # ''for kmax=0.20, '_kbin3' for kmax=0.25
# if kbin == 'kbin2':
#     kmax     = 0.20
# elif kbin == 'kbin3':
#     kmax     = 0.25

kmin     = 0.008
kmax     = 0.20
binning  = 0.006
k_ev     = np.arange(kmin, kmax+0.001, binning)
klim     = {ell*2: (kmin,kmax,binning) for ell in range(2)}

systematic_map  = {
    'RSD': '',
    'LRG': '+LRGvsmear',
    'QSO': '+QSOvsmear',
    '1%CATAS': '+1%vcatas',
    '5%CATAS': '+5%vcatas'
    }

# the cosmology parameters
catalogue   = 'fiducial'  # fiducial, Mnu_p, Mnu_ppp -- QUIJOTE catalogue
cosmology   = 'wCDM'   # LCDM, nuCDM, nsCDM, wCDM
r_pk        = 'RSD'  # RSD, LRG, QSQ, CATAS-- systematics
CovRsf      = 25  # -- covariance rescale factor
redshift    = 1.0
model       = 'FOLPS' # Kaiser, TNS, FOLPS
systematic = systematic_map.get(r_pk, '')

# Training emulator
emulator_fn = f'./model/emulator_{cosmology}_{model}_z{redshift}.npy'
if not os.path.exists(emulator_fn):
    filename = []
    filedir = f'/Users/alain/Desktop/projectNU/main/data/kbin2/{catalogue}/{r_pk}_z{redshift}/npy/'
    for file in os.listdir(filedir):
        filename.append(filedir+file)
    covariance = filedir+'*'
    # The template and theory
    cosmo = DESI()
    template = DirectPowerSpectrumTemplate(z=redshift,apmode='qisoqap', fiducial='DESI')
    if cosmology == 'nuCDM':
        template.init.params['m_ncdm'].update(fixed=False, latex=r'M_\nu', prior = {'limits': [0.0,1.0]}) #we unfix the parameter m_ncdm(Mnu)
    elif cosmology == 'nsCDM':
        template.init.params['n_s'].update(fixed=False)
    elif cosmology == 'wCDM':
        template.init.params['w0_fld'].update(fixed=False)
        # template.init.params['n_s'].update(fixed=True,value=0.9624)
    # if model == 'Kaiser':
    #     theory = KaiserTracerPowerSpectrumMultipoles(template=template, k=k_ev)
    # elif model == 'TNS':
    #     theory = TNSTracerPowerSpectrumMultipoles(template=template, k=k_ev)
    # elif model == 'FOLPS':
    theory = FOLPSTracerPowerSpectrumMultipoles(template=template, k=k_ev)
    observable = TracerPowerSpectrumMultipolesObservable(data= filename,
                                                    covariance= covariance,
                                                    klim=klim,
                                                    theory=theory,
                                                    kin=np.arange(0.001,0.35,0.002))
    likelihood = ObservablesGaussianLikelihood(observable, scale_covariance = 1/CovRsf) #
    likelihood()
    emulator = Emulator(theory.pt, engine=TaylorEmulatorEngine(order=3, method='finite')) # Taylor expansion, up to a given order
    emulator.set_samples() # evaluate the theory derivatives (with jax auto-differentiation if possible, else finite differentiation)
    emulator.fit()
    emulator.save(emulator_fn)
    print("Training finished")

[000000.54] [0/1] 06-21 12:05  TracerPowerSpectrumMultipolesObservable INFO     Loading 100 files [/Users/alain/Desktop/projectNU/main/data/kbin2/fiducial/RSD_z1.0/npy/fiducial_122_z1.0.npy, ..., /Users/alain/Desktop/projectNU/main/data/kbin2/fiducial/RSD_z1.0/npy/fiducial_101_z1.0.npy, ..., /Users/alain/Desktop/projectNU/main/data/kbin2/fiducial/RSD_z1.0/npy/fiducial_189_z1.0.npy, ..., /Users/alain/Desktop/projectNU/main/data/kbin2/fiducial/RSD_z1.0/npy/fiducial_141_z1.0.npy, ..., /Users/alain/Desktop/projectNU/main/data/kbin2/fiducial/RSD_z1.0/npy/fiducial_180_z1.0.npy].
[000001.67] [0/1] 06-21 12:05  TracerPowerSpectrumMultipolesObservable INFO     Loading 100 files [/Users/alain/Desktop/projectNU/main/data/kbin2/fiducial/RSD_z1.0/npy/fiducial_100_z1.0.npy, ..., /Users/alain/Desktop/projectNU/main/data/kbin2/fiducial/RSD_z1.0/npy/fiducial_120_z1.0.npy, ..., /Users/alain/Desktop/projectNU/main/data/kbin2/fiducial/RSD_z1.0/npy/fiducial_140_z1.0.npy, ..., /Users/alain/Desktop/projectNU

KeyboardInterrupt: 